In [1]:
import numpy as np
from pyfinite import ffield
import galois

F = ffield.FField(7, gen=0x83, useLUT=-1)

def Exponentiate(base,power):
    ans = base
    for i in range(1,power):
        ans = F.Multiply(ans,base)
    return ans


def LinearTransform(linmat,msg):
    ans = [0]*8
    for i in range(8):
        temp  = []
        mul=[]
        
        for j in range(8):
            mul.append(F.Multiply(linmat[i][j],msg[i]))
    
        for k in range(8):
            temp.append(np.bitwise_xor(ans[k],mul[k]))
        ans = temp
    return ans

In [2]:
def dec_blk(cipher):
  plain= ""
  for i in range(0,len(cipher),2):
      plain+=chr(16*(ord(cipher[i:i+2][0]) - ord('f')) + ord(cipher[i:i+2][1]) - ord('f'))
  return plain

In [4]:
#for diagonal elements
PossibleExponents = [[] for i in range(8)]   
possibleDiagonalVals=[[[] for i in range(8)] for j in range(8)]
input_file = open('plaintexts.txt','r')
output_file = open('ciphertexts.txt','r')
input = (input_file.readlines()[0]).strip().split(' ')
output = output_file.readlines()

input_string = []
for msg in input:
    input_string.append(dec_blk(msg)[0])

output_string = []
for i in range(len(output)):
    x = []
    for msg in output[i].strip().split(' '):
        x.append(dec_blk(msg)[i])
    output_string.append(x)
#print(output_string)
for k in range(8):
    for i in range(1, 127):
        for j in range(1, 128):
          flag = True
          for m in range(128):
            if(ord(output_string[k][m]) != Exponentiate(F.Multiply(Exponentiate(F.Multiply(Exponentiate(ord(input_string[m]), i), j), i), j), i)):
              flag = False
              break
          if(flag):
            PossibleExponents[k].append(i)
            possibleDiagonalVals[k][k].append(j)
print("Possible diagonal values: \n")
print(possibleDiagonalVals)
print("\n\nPossible exponents: \n")
print(PossibleExponents)


output_string = []
for i in range(len(output)-1):
    x = []
    for msg in output[i].strip().split(' '):
        x.append(dec_blk(msg)[i+1])
    output_string.append(x)

for ind in range(7):
  for i in range(1, 128):
      for p1, e1 in zip(PossibleExponents[ind+1], possibleDiagonalVals[ind+1][ind+1]):
          for p2, e2 in zip(PossibleExponents[ind], possibleDiagonalVals[ind][ind]):
              for k in range(128):
                  flag = True
                  x1 = F.Multiply(Exponentiate(F.Multiply(Exponentiate(ord(input_string[k]), p2), e2), p2), i)
                  x2 = F.Multiply(Exponentiate(F.Multiply(Exponentiate(ord(input_string[k]), p2), i), p1), e1)
                  c1 = np.bitwise_xor(x1,x2)
                  if(ord(output_string[ind][k]) != Exponentiate(c1,p1)):
                      flag = False
                      break
              if flag:
                  PossibleExponents[ind+1] = [p1]
                  possibleDiagonalVals[ind+1][ind+1] = [e1]
                  PossibleExponents[ind] = [p2]
                  possibleDiagonalVals[ind][ind] = [e2]
                  possibleDiagonalVals[ind][ind+1] = [i]
print('\n\n===========================\n\n')
print("Diagonal values: \n")
print(possibleDiagonalVals)
print("\n\nExponents: \n")
print(PossibleExponents)


Possible diagonal values: 

[[[73, 84, 20], [], [], [], [], [], [], []], [[], [122, 62, 70], [], [], [], [], [], []], [[], [], [119, 43, 5], [], [], [], [], []], [[], [], [], [12, 52, 100], [], [], [], []], [[], [], [], [], [5, 31, 112], [], [], []], [[], [], [], [], [], [11, 122, 100], [], []], [[], [], [], [], [], [], [52, 27, 27], []], [[], [], [], [], [], [], [], [38, 65, 52]]]


Possible exponents: 

[[18, 21, 88], [26, 113, 115], [2, 38, 87], [71, 79, 104], [78, 85, 91], [45, 93, 116], [1, 19, 107], [22, 37, 68]]




Diagonal values: 

[[[84], [113], [], [], [], [], [], []], [[], [70], [30], [], [], [], [], []], [[], [], [43], [1], [], [], [], []], [[], [], [], [12], [109], [], [], []], [[], [], [], [], [112], [98], [], []], [[], [], [], [], [], [11], [95], []], [[], [], [], [], [], [], [27], []], [[], [], [], [], [], [], [], [38, 65, 52]]]


Exponents: 

[[21], [115], [38], [71], [91], [45], [19], [22, 37, 68]]


In [5]:
def EAEAE (msg, lin_mat, exp_mat):
  msg = [ord(m) for m in msg]
  exponents = [Exponentiate(msg[i], exp_mat[i]) for i in range(len(msg))]
  linear_transformed = LinearTransform(lin_mat, exponents)
  exponents_2 = [Exponentiate(linear_transformed[i], exp_mat[i]) for i in range(len(linear_transformed))]
  linear_transformed_2 = LinearTransform(lin_mat, exponents_2)
  final_output = [Exponentiate(linear_transformed_2[i], exp_mat[i]) for i in range(len(linear_transformed_2))]
  return final_output
  


input_file = open('plaintexts.txt','r')
output_file = open('ciphertexts.txt','r')
input = input_file.readlines()
output = output_file.readlines()



input_string = [[dec_blk(msg) for msg in input[i].strip().split(' ')] for i in range(len(input))]
output_string = [[dec_blk(msg) for msg in output[i].strip().split(' ')] for i in range(len(output))]

for indexex in range(0,6):
    offset = indexex + 2
    
    exp_list = [e[0] for e in PossibleExponents]
    lin_trans_list = np.zeros((8,8),int)

    for i in range(8):
      for j in range(8):     
        if(len(possibleDiagonalVals[i][j]) != 0):
            lin_trans_list[i][j] = possibleDiagonalVals[i][j][0]
        else:
            lin_trans_list[i][j] = 0
    
    for index in range(8):
        if(index > (7-offset)):
            continue

        for i in range(127):
            lin_trans_list[index][index+offset] = i+1
            flag = True
            for inps, outs in zip(input_string[index], output_string[index]):
                x1 = EAEAE(inps, lin_trans_list, exp_list)[index+offset]
                x2 = outs[index+offset]
                if x1 != ord(x2):
                    flag = False
                    break
            if flag==True:
                possibleDiagonalVals[index][index+offset] = [i+1]
                
A = np.zeros((8,8),dtype='int')

for i in range(0,8):
    for j in range(0,8):
      if len(possibleDiagonalVals[j][i]) != 0:
       A[i][j] = possibleDiagonalVals[j][i][0]



E = exp_list

print('Linear Transformation Matrix :\n',A)
print('\n\n')
print('Exponent Vector : \n',E)

Linear Transformation Matrix :
 [[ 84   0   0   0   0   0   0   0]
 [113  70   0   0   0   0   0   0]
 [ 13  30  43   0   0   0   0   0]
 [100  23   1  12   0   0   0   0]
 [110  34   1 109 112   0   0   0]
 [ 25  44  28  53  98  11   0   0]
 [  2 122  23 103  27  95  27   0]
 [ 65   2  86  27  15  66   0  38]]



Exponent Vector : 
 [21, 115, 38, 71, 91, 45, 19, 22]


In [6]:
E_inverse = np.zeros((128, 128), dtype = int)

for base in range(0,128):
    temp = 1
    for power in range(1,127):
        result = F.Multiply(temp, base)
        E_inverse[power][result] = base
        temp = result
    
GF = galois.GF(2**7)
A = GF(A)
invA = np.linalg.inv(A)

In [7]:
password = "lhgomlmjmohhmkmpjqkmksmngnglflif" #Encrypted password
GF = galois.GF(2**7)

def E_inv(block, E):
    new_list = []
    for i in range(8):
        new_list.append(E_inverse[E[i]][block[i]])
    return new_list

def A_inv(block, A):
    return np.matmul(GF(A),GF(block))

dec_pass = ""
for i in range(0,2): 
    elements = password[16*i:16*(i+1)]
    currentBlock = []
    currentBlock += [(ord(elements[j]) - ord('f'))*16 + (ord(elements[j+1]) - ord('f')) for j in range(0, 15, 2)]

    EAEAE = E_inv(A_inv(E_inv(A_inv(E_inv(currentBlock, E), invA),E), invA),E)
    for ch in EAEAE:
        dec_pass += chr(ch)
    
print("\n\nPassword is",dec_pass)



Password is utomcrlsfj000000
